# Explore and cluster the neighborhoods in Toronto Part - 1 and 2
### Scrape Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


# Option A
### We can use BeautifulSoup


# Option B 
### Use python pandas 

### visit the web page "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
##### Explore the web page
##### It looks not very complicated so python pandas can be used to read & load the data

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikipage = pd.read_html(url)



# Explore Data and its struture

In [3]:
type(wikipage)

list

In [4]:
len(wikipage)

3

### Check the every element of list 'wikipage[0], 'wikipage[1], 'wikipage[2] one by one

In [5]:
wikipage[0].head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
wikipage[1].head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,NaN,Canadian postal codes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL,NS,PE,NB,QC,ON,MB,SK,AB,...,L,M,N,P,R,S,T,V,X,Y
2,NL,NS,PE,NB,QC,ON,MB,SK,AB,BC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,B,C,E,G,H,J,K,L,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
wikipage[2].head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NL,NS,PE,NB,QC,ON,MB,SK,AB,BC,NU/NT,YT,NaN,NaN,NaN,NaN,NaN,NaN
1,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


# Data Selection

### Now it is clear wikipage is a nested list structure and our data is in "wikipage[0]"
#### Create a data frame Totonto_df with wikipage[0]

In [8]:
Totonto_df = wikipage[0]
Totonto_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### First row is holds the name of columns 
#### so we will make it the column name of daraframe  and delete the first row

In [9]:
Totonto_df[0:1].values.tolist()

[['Postcode', 'Borough', 'Neighbourhood']]

In [10]:
Totonto_df.columns=['Postcode','Borough','Neighborhood']

#### Remove first row it holds the name of column and reset the index

In [11]:
#Totonto_df = Totonto_df.iloc[1:]
Totonto_df.drop(Totonto_df.index[:1], inplace=True)
Totonto_df.reset_index(drop=True, inplace=True)
Totonto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
Totonto_df.shape

(289, 3)

# Data Preparation


### Target 1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [13]:
Totonto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Target 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [14]:
Totonto_df.replace({'Borough': 'Not assigned' }, np.nan, inplace = True)
Totonto_df.head()
# simply drop whole row with NaN in "price" column
Totonto_df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped two rows
Totonto_df.reset_index(drop=True, inplace=True)
Totonto_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [15]:
Totonto_df.shape

(212, 3)

### Target 3. More than one neighborhood can exist in one postal code area. 
#### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [16]:
# Groupby and join can be used for the purpose
Totonto_df = Totonto_df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
Totonto_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
Totonto_df.shape

(103, 3)

### Target 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [18]:
#Totonto_df[ (Totonto_df.Neighbourhood=='Not assigned')]['Neighborhood'] = [Borough]
Totonto_df['Neighborhood'] = np.where(Totonto_df['Neighborhood'] == 'Not assigned', Totonto_df['Borough'], Totonto_df['Neighborhood'])
Totonto_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Target 5. Use the .shape method to print the number of rows of your dataframe.

In [19]:
Totonto_df.shape

(103, 3)

# Explore and cluster the neighborhoods in Toronto Part - 2

In [20]:
!conda install -c conda-forge geopy --yes # geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.63 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.52 MB/s
Libraries imported.


## Target - Use the Geocoder package or the csv file to create the dataframe:
#### Postcode, Borough, Neighbourhood, Latitude, Longitude
#### Option 1 - Geocoder
#### Option 2 - csv File

## Option 1(A) - Geocoder

# Issues - 
#### 1. Post code is assigned to more then one Neighborhood
#### 2. Borough to more then one post code
#### both above condition adds complexity

In [21]:

# M5J 	Downtown Toronto 	Harbourfront East, Toronto Islands, Union Station 	43.640816 	-79.381752
nbour='Harbourfront East'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(nbour)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood {} are {}, {}.'.format(nbour,latitude, longitude))

nbour='Toronto Islands'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(nbour)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood {} are {}, {}.'.format(nbour,latitude, longitude))

nbour='Union Station'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(nbour)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood {} are {}, {}.'.format(nbour,latitude, longitude))

nbour='Downtown Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(nbour)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood {} are {}, {}.'.format(nbour,latitude, longitude))

nbour='M5J'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(nbour)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood {} are {}, {}.'.format(nbour,latitude, longitude))

#nbour='Highland Creek, Rouge Hill, Port Union'
#geolocator = Nominatim(user_agent="ny_explorer")
#location = geolocator.geocode(nbour)
#latitude = location.latitude
#longitude = location.longitude
#print('The geograpical coordinate of Neighburhood {} are {}, {}.'.format(nbour,latitude, longitude))


The geograpical coordinate of Neighborhood Harbourfront East are 1.2654508, 103.8226522.
The geograpical coordinate of Neighborhood Toronto Islands are 43.6161962, -79.3693582.
The geograpical coordinate of Neighborhood Union Station are 43.64469, -79.3799656881095.
The geograpical coordinate of Neighborhood Downtown Toronto are 43.6541737, -79.3808116451341.
The geograpical coordinate of Neighborhood M5J are 43.6391759881618, -79.3830785001228.


## Option 1(B) - Geopy - Nominatim

In [22]:
to_post_ll_df = pd.DataFrame(columns=['Postcode', 'Latitude', 'Longitude'])
bro = ''
for index, row in Totonto_df.iterrows():
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(row['Borough'] + ' Canada')
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of  {} are {}, {}.'.format(row['Postcode'],latitude, longitude))
    to_post_ll_df=to_post_ll_df.append([{'Postcode':row['Postcode'],'Latitude':latitude,'Longitude':longitude}], ignore_index=True)
to_post_ll_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.773077,-79.257774
1,M1C,43.773077,-79.257774
2,M1E,43.773077,-79.257774
3,M1G,43.773077,-79.257774
4,M1H,43.773077,-79.257774


## Option 2 - csv File
### 1. DownLoad data 

In [23]:
path = 'https://cocl.us/Geospatial_data'
g_df = pd.read_csv(path)
g_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 2. Merage dataframe on Postcode
### 1. Rename the column 'Postal Code' To 'Postcode'
#### 2. Merge Totonto_df with g_df on 'PostCode'
#### 3. (how = 'inner' is used )

In [24]:
g_df = g_df.rename(columns={'Postal Code': 'Postcode'})
Totonto_Geo_df = pd.merge(Totonto_df, g_df, how='inner', on=['Postcode'])

In [25]:
Totonto_Geo_df.shape

(103, 5)

In [26]:
Totonto_Geo_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


###  Only boroughs that contain the word Toronto

In [27]:
Totonto_Geo_df[Totonto_Geo_df['Borough'].str.contains('Toronto')]
Totonto_Geo_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
